In [1]:
!pip install pandas
import pandas as pd
import os
import re # For processing age strings

# --- Configuration ---
original_csv_name = "survey_data_updated 5.csv"
# List of multi-select columns to process and their desired output file names
multi_select_columns_config = {
    'LanguageHaveWorkedWith': 'top_languages_used.csv',
    'DatabaseHaveWorkedWith': 'top_databases_used.csv', # CORRECTED: Changed 'DatabaseWorkedWith' to 'DatabaseHaveWorkedWith'
    'PlatformHaveWorkedWith': 'top_platforms_used.csv',
    'WebframeHaveWorkedWith': 'top_web_frameworks_used.csv',
    # Corrected "Desired Next Year" column names based on your provided df.columns.tolist()
    'LanguageWantToWorkWith': 'top_languages_desired.csv',
    'DatabaseWantToWorkWith': 'top_databases_desired.csv',
    'PlatformWantToWorkWith': 'top_platforms_desired.csv',
    'WebframeWantToWorkWith': 'top_web_frameworks_desired.csv',
}

# Number of top items to select for each category
top_n_items = 10

# Output file name for the cleaned main demographics data
demographics_output_csv = 'survey_demographics_cleaned.csv'

# --- Data Loading ---
try:
    df = pd.read_csv(original_csv_name)
    print(f"Successfully loaded '{original_csv_name}'. Shape: {df.shape}")
    print(f"Columns available: {df.columns.tolist()}")
except FileNotFoundError:
    print(f"Error: '{original_csv_name}' not found. Please ensure the file is in the correct directory.")
    print("Attempting to download from a common course data URL as a fallback...")
    try:
        # Fallback URL - adjust if your file comes from a different source
        common_file_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/HLOosvsPgIwt5dgOOh1RSg/survey-data-updated.csv"
        df = pd.read_csv(common_file_url)
        df.to_csv(original_csv_name, index=False) # Save it locally for future runs
        print(f"Successfully downloaded and loaded '{original_csv_name}' from URL.")
    except Exception as e:
        print(f"Failed to download from URL: {e}. Please ensure the file is accessible or manually place it.")
        exit() # Exit if unable to load data


# --- Helper Function to Process Multi-Select Columns ---
def process_multi_select_column(dataframe, column_name, output_csv_name, n=10):
    """
    Processes a multi-select column by splitting, stacking, counting frequencies,
    and saving the top N items to a new CSV file.
    Includes a 'Rank' column for Looker Studio.
    """
    if column_name not in dataframe.columns:
        print(f"WARNING: Column '{column_name}' not found in the DataFrame. Skipping processing.")
        return

    print(f"\nProcessing column: '{column_name}'...")

    # Handle Missing Values: Drop rows with NaN values in the specified column.
    cleaned_series = dataframe[column_name].dropna()

    if cleaned_series.empty:
        print(f"No non-null data in '{column_name}'. Skipping CSV creation for this column.")
        return

    # Split and Stack: Split the string by ';' delimiter and stack them.
    all_individual_items = cleaned_series.str.split(';', expand=True).stack()

    # Count Frequencies: Use value_counts() to get the popularity of each technology.
    item_counts = all_individual_items.value_counts()

    # Select Top N: Get the top N most frequent technologies.
    top_n_items_series = item_counts.nlargest(n)

    if top_n_items_series.empty:
        print(f"No top {n} items found in '{column_name}' after processing. Skipping CSV creation.")
        return

    # Convert to DataFrame for saving
    output_df = top_n_items_series.reset_index()
    output_df.columns = ['Item', 'Count'] # Rename columns for clarity

    # Add a 'Rank' column based on 'Count'
    output_df['Rank'] = output_df['Count'].rank(method='min', ascending=False).astype(int)
    output_df = output_df.sort_values(by='Rank') # Sort by rank for consistency

    # Save to CSV
    output_df.to_csv(output_csv_name, index=False)
    print(f"Successfully created '{output_csv_name}' with top {n} items for '{column_name}'.")
    # print(output_df.head()) # Uncomment to see head of each generated CSV

# --- Helper Function to Parse Age Strings to Consistent Categories ---
def parse_age_string_for_demographics(age_str):
    """
    Parses age strings (e.g., '25-34 years old') into a consistent categorical format (e.g., '25-34').
    Handles 'Under 18', '65 or older', and 'Prefer not to say'.
    """
    if pd.isna(age_str):
        return None
    age_str = str(age_str).strip().lower()

    if 'under 18' in age_str:
        return 'Under 18'
    elif '18-24' in age_str:
        return '18-24'
    elif '25-34' in age_str:
        return '25-34'
    elif '35-44' in age_str:
        return '35-44'
    elif '45-54' in age_str:
        return '45-54'
    elif '55-64' in age_str:
        return '55-64'
    elif '65 years or older' in age_str:
        return '65+'
    elif 'prefer not to say' in age_str:
        return 'Prefer not to say'
    else:
        # Fallback for any other unexpected formats, return as is or None
        return age_str # Looker Studio can handle these as distinct categories


# --- Main Data Processing for Dashboards ---

# 1. Process all multi-select columns
print("\n--- Processing Multi-Select Columns ---")
for col_name, output_file in multi_select_columns_config.items():
    process_multi_select_column(df, col_name, output_file, n=top_n_items)

# 2. Prepare main demographics data (including processed Age)
print("\n--- Preparing Demographics Data ---")
if 'Age' in df.columns:
    df['Age_Grouped'] = df['Age'].apply(parse_age_string_for_demographics)
    print(f"Created 'Age_Grouped' column. Unique values (first 10): {df['Age_Grouped'].dropna().unique().tolist()[:10]}")
else:
    print("WARNING: 'Age' column not found in original DataFrame. Age-related demographics may be affected.")

# Select relevant columns for the demographics dashboard.
# Ensure these columns exist in your original CSV.
demographics_columns = [
    'ResponseId', # Keep ResponseId for counting distinct users if needed
    'Age_Grouped', # The new processed age column
    'Country',
    'EdLevel',
    'Employment'
]

# Filter out only the columns we need for demographics and drop rows with NaNs in critical demographic columns
# Adjust `subset` as per which columns are absolutely critical for all demographic panels
df_demographics_cleaned = df[demographics_columns].dropna(subset=['Age_Grouped', 'Country', 'EdLevel', 'Employment'])

# Save the cleaned demographics DataFrame to a new CSV
df_demographics_cleaned.to_csv(demographics_output_csv, index=False)
print(f"\nSuccessfully created '{demographics_output_csv}' for demographics dashboards. Shape: {df_demographics_cleaned.shape}")
print(df_demographics_cleaned.head())


print("\n--- All Data Preparation Complete ---")
print("Please upload the following CSV files to Google Looker Studio:")
for output_file in multi_select_columns_config.values():
    if os.path.exists(output_file):
        print(f"- {output_file}")
    else:
        print(f"- {output_file} (Not created, check warnings above)")
if os.path.exists(demographics_output_csv):
    print(f"- {demographics_output_csv}")
else:
    print(f"- {demographics_output_csv} (Not created, check warnings above)")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 162.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 188.9 MB/s eta 0:00:00
Successfully loaded 'survey_data_updated 5.csv'. Shape: (18845, 114)
Columns available: ['ResponseId', 'MainBranch', 'Age', 'Employment', 'RemoteWork', 'Check', 'CodingActivities', 'EdLevel', 'LearnCode', 'LearnCodeOnline', 'TechDoc', 'YearsCode', 'YearsCodePro', 'DevType', 'OrgSize', 'PurchaseInfluence', 'BuyNewTool', 'BuildvsBuy', 'TechEndorse', 'Country', 'Currency', 'CompTotal', 'LanguageHaveWorkedWith', 'LanguageWantToWorkWith', 'LanguageAdmired', 'DatabaseHaveWorkedWith', 'DatabaseWantToWorkWith', 'DatabaseAdmired', 'PlatformHaveWorkedWith', 'PlatformWantToWorkWith', 'PlatformAdmired', 'WebframeHaveWorkedWith', 'WebframeWantToWorkWith', 'WebframeAdmired', 'EmbeddedHaveWorkedWith', 'EmbeddedWantToWorkWith', 'EmbeddedAdmired', 'MiscTechHaveWorkedWith', 'MiscTechWantToWorkWith', 'MiscTechAdmired', 'ToolsTechHaveWorkedWith', '